In [84]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

In [85]:
batch_size = 15
num_classes = 12
epochs = 12

# input image dimensions
img_rows, img_cols = 128, 128

# the data, split between train and test sets

##### import image dataset #####
import os
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from collections import defaultdict
from sklearn.model_selection import train_test_split

cate_label=[]
DATASET_LOCATION = './gear_images_new'
imglabel_db = defaultdict(list)
for imgclass in os.listdir(DATASET_LOCATION):
    imgclass_path = DATASET_LOCATION + "/" + imgclass
    imgclass_arr = []
    cate_label.append(imgclass)
    for filename in os.listdir(imgclass_path):
        
        img_path = imgclass_path + "/" + filename

        #print(img_path)
        #imgcount += 1
        img = Image.open(img_path)
        #convert images to RGB arrays
        img_arr = np.array(img)
        imgclass_arr.append(img_arr) #no need to flatten, retaint the shape
        img.close()

    imglabel_db[imgclass]=imgclass_arr
    
    
def datasplit(database, testSize):
    #divide image data (X) and labels (y)

    X = []
    y = []
   
    for label in database:
        for imgdata in database[label]:
            X.append(imgdata)
            y.append(label)
            

    #split X and y into train and test

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, random_state =42)
    return X_train, y_train, X_test, y_test


In [86]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(cate_label)
X_train, y_train, X_test, y_test = datasplit(imglabel_db, 0.2)
y_test=le.transform(y_test)
y_train=le.transform(y_train)


X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_train.shape[0], 'test samples')

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
# binary_crossentropy


x_train shape: (1697, 128, 128, 3)
1697 train samples
1697 test samples


In [88]:
input_shape= (128,128,3)

### build model
model = Sequential()


## change parameters and layer
model.add(Conv2D(20, (5, 5), padding='valid', activation='relu', input_shape=input_shape, data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(50, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
# model.add(Dropout(0.5))
model.add(Dense(12))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))


Train on 1697 samples, validate on 425 samples
Epoch 1/12
1697/1697 [==============================] - 34s 20ms/step - loss: 1.3891 - acc: 0.6070 - val_loss: 9.7493 - val_acc: 0.1741
Epoch 2/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.4570 - acc: 0.8533 - val_loss: 1.7491 - val_acc: 0.4941
Epoch 3/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.2590 - acc: 0.9169 - val_loss: 0.3638 - val_acc: 0.8800
Epoch 4/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.1566 - acc: 0.9452 - val_loss: 1.5153 - val_acc: 0.7412
Epoch 5/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.1215 - acc: 0.9658 - val_loss: 0.6161 - val_acc: 0.8612
Epoch 6/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.0841 - acc: 0.9758 - val_loss: 0.4476 - val_acc: 0.9200
Epoch 7/12
1697/1697 [==============================] - 34s 20ms/step - loss: 0.0896 - acc: 0.9806 - val_loss: 0.4559 - val_acc: 0.9200
E

In [89]:
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5327616834289888
Test accuracy: 0.927058823669658
